<a href="https://colab.research.google.com/github/ElyasBelkhir/VocalVault/blob/main/VoiceAuthentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets transformers speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.8/553.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 31.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create function to extract speaker embeddings

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
from transformers import Wav2Vec2FeatureExtractor
from transformers import WavLMForXVector

# Extract embedding vector for each audio sample using pre-trained
# model for speaker verification (WavLM)
device = "cuda" if torch.cuda.is_available() else "cpu"
feature_extractor_wav2vec = Wav2Vec2FeatureExtractor.from_pretrained(
    "microsoft/wavlm-base-plus-sv")
model_wav_lm = WavLMForXVector.from_pretrained(
    "microsoft/wavlm-base-plus-sv").to(device)

N_SECONDS_SEGMENT = 4
SAMPLING_RATE = 16000

def extract_embeddings(model, feature_extractor, data, device):
    """Use WavLM model to extract embeddings for audio segments"""
    emb_train = list()
    for i in tqdm(range(len(data))):
        # Add padding to ensure compatibility
        padded_data = np.pad(data[i], (0, max(0, SAMPLING_RATE * N_SECONDS_SEGMENT - data[i].shape[0])), 'constant')
        inputs = feature_extractor(
            padded_data,
            sampling_rate=SAMPLING_RATE,
            return_tensors="pt",
            padding="longest"
        ).to(device)
        with torch.no_grad():
            embeddings = model(**inputs).embeddings

        # Normalize embeddings along the last dimension (dimension -1)
        embeddings_normalized = torch.nn.functional.normalize(embeddings, dim=-1)

        emb_train += torch.nn.functional.normalize(
            embeddings.cpu(), dim=-1).cpu()

    return torch.stack(emb_train)

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/58.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/wavlm-base-plus-sv were not used when initializing WavLMForXVector: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at microsoft/wavlm-base-plus-sv and are newly initialized: ['wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a d

Create function to load wav file

In [ ]:
import librosa

def load_wav_file(file_path, sampling_rate):
    """ Load a WAV file. """
    audio, _ = librosa.load(file_path, sr=sampling_rate)
    return audio

Return whether 2 wav files are same or not

In [ ]:
def compare_audio(wav_1, wav_2):
  # Load your WAV files
  wav_file_1 = load_wav_file(wav_1, SAMPLING_RATE)
  wav_file_2 = load_wav_file(wav_2, SAMPLING_RATE)

  # Extract embeddings
  x_wav_files = [wav_file_1, wav_file_2]
  embeddings = extract_embeddings(model_wav_lm, feature_extractor_wav2vec, x_wav_files, device)

  embedding_1 = embeddings[0]
  embedding_2 = embeddings[1]

  cosine_sim = torch.nn.CosineSimilarity(dim=-1)
  similarity = cosine_sim(embedding_1, embedding_2)
  threshold = 0.86  # the optimal threshold is dataset-dependent
  if similarity < threshold:
    print("\nSpeakers are not the same")
  else:
    print("\nSpeakers are the same")

In [ ]:
compare_audio("/content/drive/MyDrive/Audio/ibby2.wav", "/content/drive/MyDrive/Audio/ibby.wav")

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 2/2 [00:08<00:00,  4.48s/it]


Speakers are the same
